<a href="https://colab.research.google.com/github/Sg-11004/GENAI_LAB_PROJECT/blob/main/MOVIE_RECOMMEDATION_USING_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install langchain gradio sentence-transformers transformers faiss-cpu langchain-community


In [26]:
import pandas as pd

# Load the dataset (replace with the actual path if necessary)
dataset_path = r"/content/movies_metadata.csv"
movies_df = pd.read_csv(dataset_path)

# Check for null values in each column
print(movies_df.isnull().sum())

# Data Cleaning
# Drop unnecessary columns (if needed)
columns_to_drop = ['homepage', 'tagline','belongs_to_collections','poster_path']
movies_df = movies_df.drop(columns=columns_to_drop, errors='ignore')

# Handle missing values (example: drop rows with missing titles or ratings)
movies_df = movies_df.dropna(subset=['title', 'vote_average'])

# Convert columns to appropriate data types (if needed)
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Feature Extraction (e.g., extracting genres from lists)
def extract_genres(genres_str):
    try:
        genres = eval(genres_str)  # Converts string to list
        return ", ".join([genre['name'] for genre in genres]) if genres else ""
    except:
        return ""

# Apply genre extraction
movies_df['genres'] = movies_df['genres'].apply(extract_genres)

# Check the cleaned data
print(movies_df.head())



<ipython-input-26-ddfad30eb3f0>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(dataset_path)


   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [27]:
print(movies_df.isnull().sum())

adult                        0
belongs_to_collection    40970
budget                       0
genres                       0
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   0
production_companies         0
production_countries         0
release_date                84
revenue                      0
runtime                    257
spoken_languages             0
status                      81
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64


In [28]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [29]:
movies_df['genres'].fillna('Unknown', inplace=True)
movies_df['original_language'].fillna(movies_df['original_language'].mode()[0], inplace=True)
movies_df['release_date'].fillna('Unknown', inplace=True)
movies_df['runtime'].fillna(movies_df['runtime'].median(), inplace=True)
movies_df['status'].fillna('Unknown', inplace=True)
movies_df['imdb_id'].fillna('Unknown', inplace=True)

# Check if there are any remaining missing values
print(movies_df.isnull().sum())

# Save the cleaned dataset
movies_df.to_csv('cleaned_movies_metadata.csv', index=False)

# Display the cleaned dataset's first few rows
print(movies_df.head())

<ipython-input-29-0dd84b20cf4e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['genres'].fillna('Unknown', inplace=True)
<ipython-input-29-0dd84b20cf4e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

adult                        0
belongs_to_collection    40970
budget                       0
genres                       0
id                           0
imdb_id                      0
original_language            0
original_title               0
overview                   954
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                      0
spoken_languages             0
status                       0
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                    

In [30]:
movies_df.isnull().sum()

,0
adult,0
belongs_to_collection,40970
budget,0
genres,0
id,0
imdb_id,0
original_language,0
original_title,0
overview,954
popularity,0


In [34]:
movies_df['popularity'] = pd.to_numeric(movies_df['popularity'], errors='coerce')

# Impute missing values in the columns empty if preferred
movies_df['overview'].fillna('No Overview', inplace=True)
movies_df['popularity'].fillna(movies_df['popularity'].median(), inplace=True)
movies_df['production_companies'].fillna('Unknown', inplace=True)
movies_df['production_countries'].fillna('Unknown', inplace=True)
movies_df['revenue'].fillna(0, inplace=True)  # or use median if more appropriate
movies_df['spoken_languages'].fillna('Unknown', inplace=True)
movies_df['title'].fillna('Unknown', inplace=True)
movies_df['video'].fillna('Unknown', inplace=True)
movies_df['vote_average'].fillna(movies_df['vote_average'].median(), inplace=True)
movies_df['vote_count'].fillna(movies_df['vote_count'].median(), inplace=True)

<ipython-input-34-bc5fa56f0b05>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['overview'].fillna('No Overview', inplace=True)
<ipython-input-34-bc5fa56f0b05>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [38]:
movies_df.drop(columns=['belongs_to_collection'],inplace=True)

In [39]:
movies_df.isnull().sum()

,0
adult,0
budget,0
genres,0
id,0
imdb_id,0
original_language,0
original_title,0
overview,0
popularity,0
production_companies,0


In [40]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

In [41]:
movies_df['text'] = movies_df['title'] + " " + movies_df['overview']


In [42]:
embeddings=HuggingFaceEmbeddings()

<ipython-input-42-5897ca6a0cdb>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
<ipython-input-42-5897ca6a0cdb>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  fro

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [49]:
from langchain.schema import Document
documents = []
for index, row in movies_df.iterrows():
    text = row['title'] + " " + row['overview']
    doc = Document(page_content=text, metadata={"title": row['title'], "overview": row['overview']})
    documents.append(doc)
documents[0]
print(len(documents))

45460


In [47]:
vector_store=FAISS.from_documents(documents, embeddings)

In [50]:
retriever=vector_store.as_retriever()

In [85]:

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import RetrievalQA

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50
)

gpt2_model = HuggingFacePipeline(pipeline=pipe)

qa_chain = RetrievalQA.from_chain_type(
    llm=gpt2_model,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

query = "What is the plot of 12 Monkeys"

response = qa_chain({"query": query})
print(response['result'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Twelve Monkeys In the year 2035, convict James Cole reluctantly volunteers to be sent back in time to discover the origin of a deadly virus that wiped out nearly all of the earth's population and forced the survivors into underground communities. But when Cole is mistakenly sent to 1990 instead of 1996, he's arrested and locked up in a mental hospital. There he meets psychiatrist Dr. Kathryn Railly, and patient Jeffrey Goines, the son of a famous virus expert, who may hold the key to the mysterious rogue group, the Army of the 12 Monkeys, thought to be responsible for unleashing the killer disease.

The Hamster Factor and Other Tales of Twelve Monkeys A documentary following Terry Gilliam through the creation of "12 Monkeys"

She Monkeys When Emma meets Cassandra, they initiate a relationship filled with physical and psychol